### Week 2, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=9470.25262, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 176.58246 each.], portfolio_value_time_series=[('2025-04-08 12:48:38', 9998.94262), ('2025-04-08 12:48:55', 9998.94262), ('2025-04-08 13:15:52', 9994.50262), ('2025-04-08 13:16:32', 9995.222619999999)])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 8960.16448, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 176.58246, "timestamp": "2025-04-08 12:48:37", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 170.02938, "timestamp": "2025-04-08 15:20:17", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-04-08 12:48:38", 9998.94262], ["2025-04-08 12:48:55", 9998.94262], ["2025-04-08 13:15:52", 9994.50262], ["2025-04-08 13:16:32", 9995.222619999999], ["2025-04-08 15:20:18", 9978.304479999999]], "total_portfolio_value": 9978.304479999999, "total_profit_loss": -21.695520000001125}'

In [5]:
account.report()

'{"name": "ed", "balance": 8960.16448, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 176.58246, "timestamp": "2025-04-08 12:48:37", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 170.02938, "timestamp": "2025-04-08 15:20:17", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-04-08 12:48:38", 9998.94262], ["2025-04-08 12:48:55", 9998.94262], ["2025-04-08 13:15:52", 9994.50262], ["2025-04-08 13:16:32", 9995.222619999999], ["2025-04-08 15:20:18", 9978.304479999999], ["2025-04-08 15:20:21", 9978.304479999999]], "total_portfolio_value": 9978.304479999999, "total_profit_loss": -21.695520000001125}'

In [6]:
account.list_transactions()

[3 shares of AMZN at 176.58246 each., 3 shares of AMZN at 170.02938 each.]

### Now we write an MCP server and use it directly!

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}, 'symbol': {'title'

In [9]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4o-mini"

In [10]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your account balance is **$8,960.16**. 

You hold **6 shares of Amazon (AMZN)**.

### Now let's build our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
result = await Runner.run(agent, request)
display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("ed")
print(context)

In [ ]:
from accounts import Account
Account.get("ed").report()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>